In [52]:
#%%writefile r1.py
import pandas as pd 

data = pd.read_csv(r"second_cars_info.csv")#导入数据

#字符串数据转换为日期
data["Boarding_time"].where(data["Boarding_time"]!="未上牌","2030年1月",inplace=True)#使用where替换
data.Boarding_time = pd.to_datetime(data["Boarding_time"],format="%Y年%m月").map(str).str.slice(0,7)
data["Boarding_time"].where(data["Boarding_time"]!="2030-01",pd.NA,inplace=True)#使用where将未上牌数值替换为空
data.sort_values("Boarding_time",inplace=True)#按照时间排序

#字符串数据转换为数值
data["Km"].where(data["Km"]!="百公里内","0.009万公里",inplace=True)#使用where替换
data.Km = data["Km"].apply(lambda x:float(x[:-3])*10000)#里程转换为数值

data["New_price"].where(data["New_price"]!="暂无","0.00万",inplace=True)#使用where替换
data.New_price = data["New_price"].apply(lambda x:float(x[:-1]))#现价转换为数值
data["New_price"].where(data["New_price"]!=0.00,pd.NA,inplace=True)#使用where将现价暂无替换空

def query(key):
    q = []
    li1 = key.split(";")#拆分检索字段
    for i in li1:
        li2 = i.split("=")#取出字段名称和筛选条件
        if li2[0] in ["Boarding_time","Km","Sec_price","New_price"]:
            li3 = li2[-1][1:-1].split(":")
            if len(li3)==1:
                q.append('(data["{}"]=={})'.format(str(li2[0]),li3[0]))
            else:
                if li3[0]=="":
                    q.append('(data["{}"]<{})'.format(str(li2[0]),li3[-1]))
                elif li3[-1]=="":
                    q.append('(data["{}"]>={})'.format(str(li2[0]),li3[0]))
                else:
                    q.append('(data["{}"]>={})'.format(str(li2[0]),li3[0]))
                    q.append('(data["{}"]<{})'.format(str(li2[0]),li3[-1]))
            
        else:
            q.append('(data["{}"].isin({}))'.format(str(li2[0]),li2[-1]))
    q = '&'.join(q)#重新拼接查询条件
    df = data[eval(q)]#查询结果
    df = pd.pivot_table(df,index=["Brand","Name"])#绘制透视表
    return df
if __name__=="__main__":
    quit = ''
    while quit!='q':
        key = str(input("查询条件"))
        try:
            df = query(key)
            print(df)
        except:
            print("没有满足条件的记录，请核对查询条件")
        quit=input("回车(继续)\q(退出)")

查询条件q
没有满足条件的记录，请核对查询条件
回车(继续)\q(退出)q


In [53]:
data

Brand                       Name Boarding_time        Km Discharge  \
1856     奔驰            奔驰S级 1995款 S320       1997-12   80000.0        欧1   
1790     本田  雅阁 1999款 VTi-E 2.3L 自动豪华版       2000-07  280000.0        国2   
5060     大众        帕萨特 2001款 1.8GLI手动型       2001-09  180000.0        国2   
1752     本田  雅阁 2001款 VTi-E 2.3L 自动豪华版       2002-02  160000.0        国2   
5066     大众          帕萨特 2001款 1.8T 自动       2002-03  110000.0        国2   
...     ...                        ...           ...       ...       ...   
8920   玛莎拉蒂      Ghibli 2017款 3.0T 标准型          <NA>      90.0        欧5   
8961   玛莎拉蒂     Levante 2016款 3.0T 标准型          <NA>    1000.0        欧5   
8962   玛莎拉蒂     Levante 2016款 3.0T 标准型          <NA>    1000.0        欧5   
10101   特斯拉          MODEL X 2016款 90D          <NA>     500.0        --   
10245   沃尔沃   沃尔沃S90 2017款 2.0T T4 智远版          <NA>      90.0        国5   

       Sec_price  New_price  
1856       19.99     130.26  
1790        1.30      32.35  
5060        0.98      24.86  
1752        1.76      32.89  
5066        1.15      26.27  
...          ...        ...  
8920       78.00      98.89  
8961       98.80     108.53  
8962       98.80     108.53  
10101      89.80     106.89  
10245      34.30      44.16  

[11281 rows x 7 columns]

In [51]:
%%writefile r2.py
import pandas as pd
from scipy import stats
import matplotlib.pyplot as plt
import matplotlib
matplotlib.rcParams['axes.unicode_minus']=False #显示负号
plt.rcParams['font.sans-serif']=['SimHei'] #中文显示

data = pd.read_csv(r"US_Crime_Rates_1960_2014.csv")#读取数据文件
data = data[(data["Year"]>=1991)]#取1991以后数据
Population=0
for k in data.columns[1:]:#遍历各犯罪字段
    x = data["Year"]
    y = data[k]
    slope,intercept,r,p,std_err=stats.linregress(x,y)#回归拟合，取模型参数
    def myfunc(x):#回归函数
        return slope*x+intercept
    mymodel = list(map(myfunc,x))
    plt.scatter(x,y)#绘制散点图
    plt.plot(x,mymodel)#回归线
    plt.title("{}线性回归".format(k))#标题
    plt.xlabel("年份")#x轴标题
    plt.grid()#网格线
    plt.show()#绘图
    num = int(myfunc(2019))#预测值
    if k == "Population":
        print("2019年{}预测值：".format(k),num)
        Population = num#2019人数预测
    else:
        print("2019年Population预测值：{}".format(Population))#2019人数预测
        print("2019年{}预测值：".format(k),num)#预测值
        print("2019年{}犯罪率预测值：{}%".format(k,round(100*num/Population,4)))#计算犯罪率
        

Writing r2.py
